In [ ]:
!pip install transformers
!pip install faiss-cpu
!pip install faiss-gpu
!pip install -U bitsandbytes
!pip install qwen_vl_utils
!pip install pandas
!pip install  torchvision
!pip install accelerate
!pip install chromadb
!pip install sentence_transformers
!pip install rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 48.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7

In [ ]:
import torch
import sklearn
from torch import nn
from torchvision import transforms
from PIL import Image

In [ ]:
import re
def preprocess_text(result):
  matches = re.findall(r'\b(yes|no)\b', result, flags=re.IGNORECASE)
  final_answer = matches[-1].lower() if matches else "None"
  return final_answer
def answer_to_number(result):
  if result == "yes":
    return 1
  elif result == "no":
    return 0
def computation(labels,results):
  FN,TN,FP,TP,accur = 0,0,0,0,0
  for i in range(len(labels)):
     if labels[i] == 1 and results[i] == 1:
       TP += 1
     elif labels[i] == 1 and results[i] == 0:
       FN += 1
     elif labels[i] == 0 and results[i] == 1:
       FP += 1
     else:
       TN += 1
  for i in range(len(labels)):
    if labels[i] == results[i]:
      accur += 1
  accuracy = accur/len(labels)
  LR_PLUS = (TP/(TP+FN))/(FP/(FP+TN))
  LR_MINUS = (FN/(TP+FN))/(TN/(FP+TN))
  NPV = TN/(TN+FN)
  answer = {
      "LR+":LR_PLUS,
      "LR-":LR_MINUS,
      "NPV":NPV,
      "accuracy":accuracy
  }
  return answer

In [ ]:
from transformers import BitsAndBytesConfig,AutoProcessor, AutoModelForVision2Seq
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
processor_idefics = AutoProcessor.from_pretrained("HuggingFaceM4/Idefics3-8B-Llama3")
model_idefics = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceM4/Idefics3-8B-Llama3",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
    device_map="auto",
)
model_idefics.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

Idefics3ForConditionalGeneration(
  (model): Idefics3Model(
    (vision_model): Idefics3VisionTransformer(
      (embeddings): Idefics3VisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(676, 1152)
      )
      (encoder): Idefics3Encoder(
        (layers): ModuleList(
          (0-26): 27 x Idefics3EncoderLayer(
            (self_attn): Idefics3VisionAttention(
              (k_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): Idefics3VisionMLP(
              (activation_fn): GELUTanh()
              (fc1): L

In [ ]:
import pandas as pd
df1 = pd.read_csv('train_offense_facts.csv', on_bad_lines='skip')
df2 = pd.read_csv('test_preprocessed.csv', on_bad_lines='skip')
df1 = df1[['id','label','only_facts']]
df2 = df2[['id','label','facts_and_arguments']]
argument_keywords = [
    'hence',
    'oppose',
    'opposes',
    'opposed',
    'opposing',
    'support',
    'supports',
    'supported',
    'supporting',
    'bailable',
    'granted',
    'rejected'
]

only_facts = []
for fact_arg in df2['facts_and_arguments']:
    sents = fact_arg.split('. ')
    new_sents = []
    for s in sents:
        flag = True
        for key in argument_keywords:
            if key in s:
                flag = False
                break
        if flag:
          new_sents.append(s)
    only_facts.append('. '.join(new_sents))
df2.loc[:, 'only_facts'] = only_facts
test_img_path = f"/content/grey.jpg"
image = Image.open(test_img_path)
image = image.resize((224, 224), Image.BICUBIC)
image = image.convert("RGB")

In [ ]:
results = []
labels = [df2['label'].iloc[i] for i in range(len(df2))]
for i in range(len(df2)):
 text = df2['only_facts'].iloc[i]
 label = df2['label'].iloc[i]

 system_prompt = '''Instruction: You are provided with the facts of a criminal case report.
                    You need to understand how a bail decision is made from the facts in order to respond to the question.'''
 user_prompt = f'''Question: The facts of the current case are the following: {text}
                  You are also given an image of the accused person which is the photo of the accused which you need to see.
                  Should the accused person be granted bail? Please answer in yes/no only.
                  \n Answer:'''

 conversation = [
     {
         "role": "system",
         "content": system_prompt
     },
    {
        "role": "user",
        "content": [
            {"type": "image", "image" : image},
            {"type": "text", "text": user_prompt}
        ]
    }
]
 text = processor_idefics.apply_chat_template(conversation, add_generation_prompt=True)
 inputs = processor_idefics(images=image, text=text, return_tensors="pt")
 inputs = inputs.to("cuda")
 generated_ids = model_idefics.generate(**inputs, return_dict_in_generate=True,
                                         output_scores=True,
                                         do_sample=True,
                                         max_new_tokens=256,
                                         temperature=0.1)
 answer_text = processor_idefics.tokenizer.batch_decode(generated_ids.sequences, skip_special_tokens=True)
 answer_text = answer_text[0].strip()
 print(i+1)
 answer_text = preprocess_text(answer_text)
 print(answer_text)
 results.append(answer_to_number(answer_text))

Streaming output truncated to the last 5000 lines.
817
no
818
no
819
no
820
no
821
no
822
no
823
no
824
no
825
no
826
no
827
no
828
no
829
no
830
no
831
no
832
no
833
no
834
no
835
yes
836
no
837
yes
838
yes
839
no
840
no
841
no
842
yes
843
yes
844
no
845
no
846
no
847
no
848
no
849
no
850
no
851
yes
852
no
853
no
854
no
855
no
856
no
857
no
858
yes
859
no
860
no
861
no
862
no
863
no
864
no
865
no
866
no
867
no
868
no
869
yes
870
yes
871
no
872
no
873
no
874
no
875
no
876
yes
877
no
878
no
879
no
880
no
881
yes
882
no
883
no
884
no
885
yes
886
no
887
no
888
no
889
no
890
no
891
no
892
no
893
no
894
yes
895
no
896
no
897
no
898
no
899
no
900
no
901
yes
902
no
903
no
904
no
905
no
906
no
907
no
908
no
909
yes
910
no
911
no
912
no
913
yes
914
no
915
no
916
no
917
no
918
yes
919
no
920
no
921
yes
922
no
923
no
924
no
925
no
926
no
927
yes
928
yes
929
yes
930
no
931
yes
932
yes
933
no
934
no
935
yes
936
no
937
no
938
no
939
no
940
yes
941
no
942
no
943
yes
944
no
945
no
946
yes
947
no
948
n

In [ ]:
print("Without RAG:")
print(labels)
print(results)
print(computation(labels,results))

Without RAG:
[np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0)

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import numpy as np
client = chromadb.Client()
collection = client.create_collection(name="docds", get_or_create=True)

embedder = SentenceTransformer("all-MiniLM-L6-v2").cuda()
docds = [
    df1['only_facts'].iloc[i]  for i in range(len(df1))
]

embeddings = embedder.encode(docds).tolist()

# Split data into smaller batches to avoid exceeding ChromaDB's batch size limit
batch_size = 5000 # Using 5000, which is less than the max_batch_size of 5461
for i in range(0, len(docds), batch_size):
    batch_docs = docds[i:i + batch_size]
    batch_embeddings = embeddings[i:i + batch_size]
    batch_ids = [f"{j}" for j in range(i, min(i + batch_size, len(docds)))]

    collection.add(
        documents=batch_docs,
        embeddings=batch_embeddings,
        ids=batch_ids
    )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def create_message(index,combined_results,combined_labels):
  return f'''case{index+1}:{combined_results[index]}\n
             Based on the facts the final decision of the case is that the bail was {'granted' if combined_labels[index]==1 else 'rejected'}.'''

In [ ]:
results = []
labels = [df2['label'].iloc[i] for i in range(len(df2))]
for i in range(len(df2)):
  results_chroma = collection.query(
    query_embeddings=embedder.encode(df2['only_facts'].iloc[i]).tolist(),
    n_results=3,
    include=["documents", "distances", "metadatas"]
)

  documents = results_chroma['documents'][0]
  sim_score = results_chroma['distances'][0]
  index=results_chroma['ids'][0]
  index = [int(index[i]) for i in range(len(index))]

  combined_labels = [df1['label'].iloc[i] for i in index]

  text = df2['only_facts'].iloc[i]
  label = df2['label'].iloc[i]
  user_prompt = '''You are a bail decision assistant. Study these example cases and apply the pattern to the new case.

EXAMPLE CASES (learn from these):

'''

# Add examples in CLEAR pattern
  for j in range(len(documents)):
    user_prompt += f'''Case {j+1}:
Facts: {documents[j]}
Decision: {"GRANT BAIL (yes)" if combined_labels[j] == 1 else "DENY BAIL (no)"}

'''

  user_prompt += f'''NEW CASE TO DECIDE:
Facts: {text}
Image: <image> of the accused

INSTRUCTIONS:
1. Compare this new case with the 3 example cases above
2. Look for SIMILAR factors between cases
3. If the new case is similar to majority of the example cases where bail was GRANTED, answer "yes"
4. If the new case is similar to majority of the example cases where bail was DENIED, answer "no"

Important: You MUST base your decision on the comparison with examples.

Decision (yes/no only): '''

  # The text is now the full user prompt including the explicit <image> token
  # We need to construct the conversation to only contain this text, not the image separately.
  conversation = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": user_prompt}
        ]
    }
]

  # Apply chat template to get the final text string for the model
  templated_text = processor_idefics.apply_chat_template(conversation, add_generation_prompt=True)

  # Pass the templated text and the actual image (as a list) to the processor
  inputs = processor_idefics(images=[image], text=templated_text, return_tensors="pt")
  inputs = inputs.to("cuda")
  generated_ids = model_idefics.generate(**inputs, return_dict_in_generate=True,
                                         output_scores=True,
                                         do_sample=True,
                                         max_new_tokens=256,
                                         temperature=0.1)
  answer_text = processor_idefics.tokenizer.batch_decode(generated_ids.sequences, skip_special_tokens=True)
  answer_text = answer_text[0].strip()
  print(i+1)
  answer_text = preprocess_text(answer_text)
  print(answer_text)
  results.append(answer_to_number(answer_text))

Streaming output truncated to the last 5000 lines.
817
no
818
no
819
no
820
no
821
no
822
no
823
yes
824
yes
825
no
826
yes
827
no
828
yes
829
no
830
no
831
no
832
no
833
no
834
no
835
yes
836
no
837
yes
838
yes
839
yes
840
no
841
yes
842
yes
843
yes
844
no
845
no
846
no
847
yes
848
yes
849
no
850
no
851
no
852
no
853
no
854
yes
855
yes
856
no
857
no
858
no
859
yes
860
no
861
no
862
no
863
yes
864
no
865
no
866
no
867
yes
868
no
869
yes
870
no
871
no
872
no
873
yes
874
no
875
yes
876
yes
877
yes
878
yes
879
yes
880
no
881
no
882
no
883
yes
884
no
885
yes
886
yes
887
yes
888
yes
889
yes
890
yes
891
no
892
no
893
yes
894
yes
895
yes
896
yes
897
no
898
no
899
no
900
no
901
yes
902
yes
903
no
904
yes
905
no
906
yes
907
yes
908
no
909
yes
910
no
911
no
912
no
913
yes
914
no
915
yes
916
no
917
no
918
yes
919
no
920
yes
921
yes
922
no
923
no
924
no
925
no
926
yes
927
yes
928
yes
929
yes
930
yes
931
yes
932
no
933
yes
934
no
935
yes
936
no
937
yes
938
no
939
yes
940
no
941
yes
942
no
943
no
94

In [ ]:
print("With RAG:")
print(labels)
print(results)
print(computation(labels,results))

With RAG:
[np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), n